In [1]:
# -*- coding: utf-8 -*-
# author：Cookly
from util import DataLoader, Features
from SBBTree_ONLINE import SBBTree
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta
import lightgbm as lgb

In [ ]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    pbar.set_description("Processing %s" % char)

In [2]:
# test code
Data = DataLoader(
				FILE_jdata_sku_basic_info='../data/jdata_sku_basic_info.csv',
				FILE_jdata_user_action='../data/jdata_user_action.csv',
				FILE_jdata_user_basic_info='../data/jdata_user_basic_info.csv',
				FILE_jdata_user_comment_score='../data/jdata_user_comment_score.csv',
				FILE_jdata_user_order='../data/jdata_user_order.csv'
			)
# train data
TrainFeatures = Features(
						DataLoader=Data,
						PredMonthBegin = datetime(2017, 4, 1),
						PredMonthEnd = datetime(2017, 4, 30),
						FeatureMonthList = [(datetime(2017, 3, 1), datetime(2017, 3, 31), 1),\
									(datetime(2017, 1, 1), datetime(2017, 3, 31), 3),\
									(datetime(2016, 10, 1), datetime(2017, 3, 31), 6)],
						MakeLabel = True
					)
# pred data
PredFeatures = Features(
					DataLoader=Data,
					PredMonthBegin = datetime(2017, 5, 1),
					PredMonthEnd = datetime(2017, 5, 31),
					FeatureMonthList = [(datetime(2017, 4, 1), datetime(2017, 4, 30), 1),\
									(datetime(2017, 2, 1), datetime(2017, 4, 30), 3),\
									(datetime(2016, 11, 1), datetime(2017, 4, 30), 6)],
					MakeLabel = False
				)

In [4]:
train_features = TrainFeatures.TrainColumns
cols = TrainFeatures.IDColumns + TrainFeatures.LabelColumns + train_features
							
###########
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
###############################################################
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)

In [5]:
# train 下个月购买次数预测 回归模型
train_features = TrainFeatures.TrainColumns
train_label_BuyNum = 'Label_30_101_BuyNum'

X = TrainFeatures.data_BuyOrNot_FirstTime[train_features].values 
y = TrainFeatures.data_BuyOrNot_FirstTime[train_label_BuyNum].values

X_pred = PredFeatures.data_BuyOrNot_FirstTime[train_features].values

model.fit(X,y)
PredFeatures.data_BuyOrNot_FirstTime[train_label_BuyNum] = model.predict(X_pred)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[1]	valid_0's auc: 0.802053	valid_0's l2: 0.749531
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.803202	valid_0's l2: 0.734705
[3]	valid_0's auc: 0.80378	valid_0's l2: 0.721143
[4]	valid_0's auc: 0.804944	valid_0's l2: 0.706859
[5]	valid_0's auc: 0.80538	valid_0's l2: 0.695459
[6]	valid_0's auc: 0.806379	valid_0's l2: 0.685605
[7]	valid_0's auc: 0.806165	valid_0's l2: 0.677238
[8]	valid_0's auc: 0.806357	valid_0's l2: 0.667606
[9]	valid_0's auc: 0.80642	valid_0's l2: 0.661401
[10]	valid_0's auc: 0.806763	valid_0's l2: 0.654651
[11]	valid_0's auc: 0.806949	valid_0's l2: 0.648821
[12]	valid_0's auc: 0.807105	valid_0's l2: 0.643409
[13]	valid_0's auc: 0.80746	valid_0's l2: 0.639002
[14]	valid_0's auc: 0.807582	valid_0's l2: 0.63566
[15]	valid_0's auc: 0.807849	valid_0's l2: 0.63245
[16]	valid_0's auc: 0.807887	valid_0's l2: 0.62989
[17]	valid_0's auc: 0.807914	valid_0's l2: 0.62636
[18]	valid_0's auc: 0.808297	valid_0's l2: 0.624059
[19]	valid_0's auc

In [6]:
###############################################################
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# train 当月首次购买时间预测 回归模型
train_label_FirstTime = 'Label_30_101_FirstTime'
X = TrainFeatures.data_BuyOrNot_FirstTime[TrainFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime]>0][train_features].values 
y = TrainFeatures.data_BuyOrNot_FirstTime[TrainFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime]>0][train_label_FirstTime].values 

X_pred = PredFeatures.data_BuyOrNot_FirstTime[train_features].values 

model.fit(X,y)
PredFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime] = model.predict(X_pred)

[1]	valid_0's l2: 60.8466
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's l2: 60.5828
[3]	valid_0's l2: 60.3303
[4]	valid_0's l2: 60.1049
[5]	valid_0's l2: 59.9017
[6]	valid_0's l2: 59.7214
[7]	valid_0's l2: 59.5546
[8]	valid_0's l2: 59.3996
[9]	valid_0's l2: 59.2614
[10]	valid_0's l2: 59.1298
[11]	valid_0's l2: 59.0009
[12]	valid_0's l2: 58.8887
[13]	valid_0's l2: 58.7755
[14]	valid_0's l2: 58.6628
[15]	valid_0's l2: 58.5743
[16]	valid_0's l2: 58.489
[17]	valid_0's l2: 58.4195
[18]	valid_0's l2: 58.356
[19]	valid_0's l2: 58.3051
[20]	valid_0's l2: 58.2535
[21]	valid_0's l2: 58.1998
[22]	valid_0's l2: 58.1456
[23]	valid_0's l2: 58.1051
[24]	valid_0's l2: 58.0604
[25]	valid_0's l2: 58.0193
[26]	valid_0's l2: 57.9966
[27]	valid_0's l2: 57.9649
[28]	valid_0's l2: 57.9415
[29]	valid_0's l2: 57.9101
[30]	valid_0's l2: 57.8796
[31]	valid_0's l2: 57.8507
[32]	valid_0's l2: 57.8167
[33]	valid_0's l2: 57.801
[34]	valid_0's l2: 57.7815
[35]	valid_0's l2: 57.7707
[36]

In [8]:
# submit
columns = ['user_id'] + [train_label_BuyNum] + [train_label_FirstTime]
out_submit = PredFeatures.data_BuyOrNot_FirstTime[columns].sort_values(['Label_30_101_BuyNum'],ascending=False)
out_submit[train_label_FirstTime] = out_submit[train_label_FirstTime].map(lambda day: datetime(2017, 5, 1)+timedelta(days=int(day+0.49-1)))

out_submit = out_submit[['user_id']+[train_label_FirstTime]]
out_submit.columns = ['user_id','pred_date']
out_submit.head(50000).to_csv('../submit/predict-baseline.csv',index=False,header=True)